In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler                      # to scale numerical value
from sklearn.impute import SimpleImputer                              # to impute missing values in data
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder     # to convert categorical feature into numerical feature
from sklearn.pipeline import Pipeline                               # to build data preprocessing pipeline
from sklearn.model_selection import train_test_split             # to split data randomly into test and train part
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error


In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
used_car = pd.read_csv("Classified Ads for Cars.csv")
used_car

,maker,model,mileage,manufacture_year,engine_displacement,engine_power,body_type,color_slug,stk_year,transmission,door_count,seat_count,fuel_type,date_created,date_last_seen,price_eur
0,ford,galaxy,151000.0,2011.0,2000.0,103.0,NaN,NaN,None,man,5,7,diesel,2015-11-14 18:10:06.838319+00,2016-01-27 20:40:15.46361+00,10584.75
1,skoda,octavia,143476.0,2012.0,2000.0,81.0,NaN,NaN,None,man,5,5,diesel,2015-11-14 18:10:06.853411+00,2016-01-27 20:40:15.46361+00,8882.31
2,bmw,NaN,97676.0,2010.0,1995.0,85.0,NaN,NaN,None,man,5,5,diesel,2015-11-14 18:10:06.861792+00,2016-01-27 20:40:15.46361+00,12065.06
3,skoda,fabia,111970.0,2004.0,1200.0,47.0,NaN,NaN,None,man,5,5,gasoline,2015-11-14 18:10:06.872313+00,2016-01-27 20:40:15.46361+00,2960.77
4,skoda,fabia,128886.0,2004.0,1200.0,47.0,NaN,NaN,None,man,5,5,gasoline,2015-11-14 18:10:06.880335+00,2016-01-27 20:40:15.46361+00,2738.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3552907,skoda,roomster,54000.0,2013.0,1200.0,63.0,other,NaN,NaN,NaN,NaN,NaN,NaN,2017-03-16 18:57:35.46558+00,2017-03-16 18:57:35.46558+00,1295.34
3552908,skoda,felicia,NaN,2000.0,NaN,50.0,other,NaN,NaN,NaN,NaN,NaN,electric,2017-03-16 18:57:37.761349+00,2017-03-16 18:57:37.761349+00,1295.34
3552909,skoda,octavia,230000.0,2006.0,1900.0,100.0,other,NaN,NaN,NaN,NaN,NaN,NaN,2017-03-16 18:57:40.435847+00,2017-03-16 18:57:40.435847+00,1295.34
3552910,skoda,fabia,NaN,2001.0,NaN,NaN,other,NaN,NaN,NaN,NaN,NaN,NaN,2017-03-16 18:57:43.595523+00,2017-03-16 18:57:43.595523+00,1295.34


In [4]:
used_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3552912 entries, 0 to 3552911
Data columns (total 16 columns):
 #   Column               Dtype  
---  ------               -----  
 0   maker                object 
 1   model                object 
 2   mileage              float64
 3   manufacture_year     float64
 4   engine_displacement  float64
 5   engine_power         float64
 6   body_type            object 
 7   color_slug           object 
 8   stk_year             object 
 9   transmission         object 
 10  door_count           object 
 11  seat_count           object 
 12  fuel_type            object 
 13  date_created         object 
 14  date_last_seen       object 
 15  price_eur            float64
dtypes: float64(5), object(11)
memory usage: 433.7+ MB


In [5]:
used_car.isna().sum()

maker                   518915
model                  1133361
mileage                 362584
manufacture_year        370578
engine_displacement     743414
engine_power            554877
body_type              1122914
color_slug             3343411
stk_year               1708156
transmission            741630
door_count              614373
seat_count              749489
fuel_type              1847606
date_created                 0
date_last_seen               0
price_eur                    0
dtype: int64

In [6]:
index = list(used_car.isna().sum().index)
values = np.round((used_car.isna().sum() / used_car.shape[0])*100, 2)

In [7]:
index

['maker',
 'model',
 'mileage',
 'manufacture_year',
 'engine_displacement',
 'engine_power',
 'body_type',
 'color_slug',
 'stk_year',
 'transmission',
 'door_count',
 'seat_count',
 'fuel_type',
 'date_created',
 'date_last_seen',
 'price_eur']

In [8]:
values

maker                  14.61
model                  31.90
mileage                10.21
manufacture_year       10.43
engine_displacement    20.92
engine_power           15.62
body_type              31.61
color_slug             94.10
stk_year               48.08
transmission           20.87
door_count             17.29
seat_count             21.10
fuel_type              52.00
date_created            0.00
date_last_seen          0.00
price_eur               0.00
dtype: float64

In [9]:
pd.DataFrame({'miss_per':values})       # percentage of missing value

,miss_per
maker,14.61
model,31.90
mileage,10.21
manufacture_year,10.43
engine_displacement,20.92
engine_power,15.62
body_type,31.61
color_slug,94.10
stk_year,48.08
transmission,20.87


In [10]:
used_car = used_car.replace('None', np.nan)
used_car

,maker,model,mileage,manufacture_year,engine_displacement,engine_power,body_type,color_slug,stk_year,transmission,door_count,seat_count,fuel_type,date_created,date_last_seen,price_eur
0,ford,galaxy,151000.0,2011.0,2000.0,103.0,NaN,NaN,NaN,man,5,7,diesel,2015-11-14 18:10:06.838319+00,2016-01-27 20:40:15.46361+00,10584.75
1,skoda,octavia,143476.0,2012.0,2000.0,81.0,NaN,NaN,NaN,man,5,5,diesel,2015-11-14 18:10:06.853411+00,2016-01-27 20:40:15.46361+00,8882.31
2,bmw,NaN,97676.0,2010.0,1995.0,85.0,NaN,NaN,NaN,man,5,5,diesel,2015-11-14 18:10:06.861792+00,2016-01-27 20:40:15.46361+00,12065.06
3,skoda,fabia,111970.0,2004.0,1200.0,47.0,NaN,NaN,NaN,man,5,5,gasoline,2015-11-14 18:10:06.872313+00,2016-01-27 20:40:15.46361+00,2960.77
4,skoda,fabia,128886.0,2004.0,1200.0,47.0,NaN,NaN,NaN,man,5,5,gasoline,2015-11-14 18:10:06.880335+00,2016-01-27 20:40:15.46361+00,2738.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3552907,skoda,roomster,54000.0,2013.0,1200.0,63.0,other,NaN,NaN,NaN,NaN,NaN,NaN,2017-03-16 18:57:35.46558+00,2017-03-16 18:57:35.46558+00,1295.34
3552908,skoda,felicia,NaN,2000.0,NaN,50.0,other,NaN,NaN,NaN,NaN,NaN,electric,2017-03-16 18:57:37.761349+00,2017-03-16 18:57:37.761349+00,1295.34
3552909,skoda,octavia,230000.0,2006.0,1900.0,100.0,other,NaN,NaN,NaN,NaN,NaN,NaN,2017-03-16 18:57:40.435847+00,2017-03-16 18:57:40.435847+00,1295.34
3552910,skoda,fabia,NaN,2001.0,NaN,NaN,other,NaN,NaN,NaN,NaN,NaN,NaN,2017-03-16 18:57:43.595523+00,2017-03-16 18:57:43.595523+00,1295.34


In [62]:
index = list(used_car.isna().sum().index)
values = np.round((used_car.isna().sum() / used_car.shape[0])*100, 2)
pd.DataFrame({'miss_per':values})       # percentage of missing value

,miss_per
maker,14.61
model,31.90
mileage,10.21
manufacture_year,10.43
engine_displacement,20.92
engine_power,15.62
body_type,31.61
color_slug,94.10
stk_year,84.91
transmission,20.87


In [63]:
clean_data = used_car.drop(['color_slug', 'fuel_type', 'stk_year'], axis=1)
clean_data

,maker,model,mileage,manufacture_year,engine_displacement,engine_power,body_type,transmission,door_count,seat_count,date_created,date_last_seen,price_eur
0,ford,galaxy,151000.0,2011.0,2000.0,103.0,NaN,man,5,7,2015-11-14 18:10:06.838319+00,2016-01-27 20:40:15.46361+00,10584.75
1,skoda,octavia,143476.0,2012.0,2000.0,81.0,NaN,man,5,5,2015-11-14 18:10:06.853411+00,2016-01-27 20:40:15.46361+00,8882.31
2,bmw,NaN,97676.0,2010.0,1995.0,85.0,NaN,man,5,5,2015-11-14 18:10:06.861792+00,2016-01-27 20:40:15.46361+00,12065.06
3,skoda,fabia,111970.0,2004.0,1200.0,47.0,NaN,man,5,5,2015-11-14 18:10:06.872313+00,2016-01-27 20:40:15.46361+00,2960.77
4,skoda,fabia,128886.0,2004.0,1200.0,47.0,NaN,man,5,5,2015-11-14 18:10:06.880335+00,2016-01-27 20:40:15.46361+00,2738.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3552907,skoda,roomster,54000.0,2013.0,1200.0,63.0,other,NaN,NaN,NaN,2017-03-16 18:57:35.46558+00,2017-03-16 18:57:35.46558+00,1295.34
3552908,skoda,felicia,NaN,2000.0,NaN,50.0,other,NaN,NaN,NaN,2017-03-16 18:57:37.761349+00,2017-03-16 18:57:37.761349+00,1295.34
3552909,skoda,octavia,230000.0,2006.0,1900.0,100.0,other,NaN,NaN,NaN,2017-03-16 18:57:40.435847+00,2017-03-16 18:57:40.435847+00,1295.34
3552910,skoda,fabia,NaN,2001.0,NaN,NaN,other,NaN,NaN,NaN,2017-03-16 18:57:43.595523+00,2017-03-16 18:57:43.595523+00,1295.34


In [64]:
clean_data.isna().sum()

maker                   518915
model                  1133361
mileage                 362584
manufacture_year        370578
engine_displacement     743414
engine_power            554877
body_type              1122914
transmission            741630
door_count             1090066
seat_count             1287099
date_created                 0
date_last_seen               0
price_eur                    0
dtype: int64

In [31]:
clean_data = clean_data.replace('None', np.nan)
clean_data

,maker,model,mileage,manufacture_year,engine_displacement,engine_power,body_type,transmission,door_count,seat_count,date_created,date_last_seen,price_eur
0,ford,galaxy,151000.0,2011.0,2000.0,103.0,NaN,man,5,7,2015-11-14 18:10:06.838319+00,2016-01-27 20:40:15.46361+00,10584.75
1,skoda,octavia,143476.0,2012.0,2000.0,81.0,NaN,man,5,5,2015-11-14 18:10:06.853411+00,2016-01-27 20:40:15.46361+00,8882.31
2,bmw,NaN,97676.0,2010.0,1995.0,85.0,NaN,man,5,5,2015-11-14 18:10:06.861792+00,2016-01-27 20:40:15.46361+00,12065.06
3,skoda,fabia,111970.0,2004.0,1200.0,47.0,NaN,man,5,5,2015-11-14 18:10:06.872313+00,2016-01-27 20:40:15.46361+00,2960.77
4,skoda,fabia,128886.0,2004.0,1200.0,47.0,NaN,man,5,5,2015-11-14 18:10:06.880335+00,2016-01-27 20:40:15.46361+00,2738.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3552907,skoda,roomster,54000.0,2013.0,1200.0,63.0,other,NaN,NaN,NaN,2017-03-16 18:57:35.46558+00,2017-03-16 18:57:35.46558+00,1295.34
3552908,skoda,felicia,NaN,2000.0,NaN,50.0,other,NaN,NaN,NaN,2017-03-16 18:57:37.761349+00,2017-03-16 18:57:37.761349+00,1295.34
3552909,skoda,octavia,230000.0,2006.0,1900.0,100.0,other,NaN,NaN,NaN,2017-03-16 18:57:40.435847+00,2017-03-16 18:57:40.435847+00,1295.34
3552910,skoda,fabia,NaN,2001.0,NaN,NaN,other,NaN,NaN,NaN,2017-03-16 18:57:43.595523+00,2017-03-16 18:57:43.595523+00,1295.34


In [32]:
clean_data.isna().sum()

maker                   518915
model                  1133361
mileage                 362584
manufacture_year        370578
engine_displacement     743414
engine_power            554877
body_type              1122914
transmission            741630
door_count             1090066
seat_count             1287099
date_created                 0
date_last_seen               0
price_eur                    0
dtype: int64

In [82]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3552912 entries, 0 to 3552911
Data columns (total 13 columns):
 #   Column               Dtype  
---  ------               -----  
 0   maker                object 
 1   model                object 
 2   mileage              float64
 3   manufacture_year     float64
 4   engine_displacement  float64
 5   engine_power         float64
 6   body_type            object 
 7   transmission         object 
 8   door_count           object 
 9   seat_count           object 
 10  date_created         object 
 11  date_last_seen       object 
 12  price_eur            float64
dtypes: float64(5), object(8)
memory usage: 352.4+ MB


In [83]:
clean_data['door_count'] = clean_data.door_count.astype('float64')
clean_data['seat_count'] = clean_data.seat_count.astype('float64')

In [84]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3552912 entries, 0 to 3552911
Data columns (total 13 columns):
 #   Column               Dtype  
---  ------               -----  
 0   maker                object 
 1   model                object 
 2   mileage              float64
 3   manufacture_year     float64
 4   engine_displacement  float64
 5   engine_power         float64
 6   body_type            object 
 7   transmission         object 
 8   door_count           float64
 9   seat_count           float64
 10  date_created         object 
 11  date_last_seen       object 
 12  price_eur            float64
dtypes: float64(7), object(6)
memory usage: 352.4+ MB


In [85]:
clean_data.corr()

,mileage,manufacture_year,engine_displacement,engine_power,door_count,seat_count,price_eur
mileage,1.000000,-0.037207,0.121294,-0.006331,0.022983,0.022569,-0.000188
manufacture_year,-0.037207,1.000000,-0.044598,0.033741,0.175522,0.033145,-0.000004
engine_displacement,0.121294,-0.044598,1.000000,0.363279,-0.053402,-0.031108,0.003932
engine_power,-0.006331,0.033741,0.363279,1.000000,-0.067178,-0.036126,0.000573
door_count,0.022983,0.175522,-0.053402,-0.067178,1.000000,0.304062,-0.003450
seat_count,0.022569,0.033145,-0.031108,-0.036126,0.304062,1.000000,-0.001692
price_eur,-0.000188,-0.000004,0.003932,0.000573,-0.003450,-0.001692,1.000000


In [86]:
clean_data.corrwith(clean_data.price_eur)

mileage               -0.000188
manufacture_year      -0.000004
engine_displacement    0.003932
engine_power           0.000573
door_count            -0.003450
seat_count            -0.001692
price_eur              1.000000
dtype: float64

In [87]:
num_features = ['mileage', 'manufacture_year', 'engine_displacement',
       'engine_power', 'door_count', 'seat_count']

cat_features = ['maker', 'model', 'body_type', 'transmission',]


In [88]:
num_data = clean_data[num_features]
num_data

,mileage,manufacture_year,engine_displacement,engine_power,door_count,seat_count
0,151000.0,2011.0,2000.0,103.0,5.0,7.0
1,143476.0,2012.0,2000.0,81.0,5.0,5.0
2,97676.0,2010.0,1995.0,85.0,5.0,5.0
3,111970.0,2004.0,1200.0,47.0,5.0,5.0
4,128886.0,2004.0,1200.0,47.0,5.0,5.0
...,...,...,...,...,...,...
3552907,54000.0,2013.0,1200.0,63.0,NaN,NaN
3552908,NaN,2000.0,NaN,50.0,NaN,NaN
3552909,230000.0,2006.0,1900.0,100.0,NaN,NaN
3552910,NaN,2001.0,NaN,NaN,NaN,NaN


In [92]:
num_data.corrwith(clean_data.price_eur)

mileage               -0.000188
manufacture_year      -0.000004
engine_displacement    0.003932
engine_power           0.000573
door_count            -0.003450
seat_count            -0.001692
dtype: float64

In [100]:
model = SimpleImputer(strategy='most_frequent')

In [101]:
num_data_tr = model.fit_transform(num_data)
num_data_tr = pd.DataFrame(num_data_tr, columns=num_features)
num_data_tr

,mileage,manufacture_year,engine_displacement,engine_power,door_count,seat_count
0,151000.0,2011.0,2000.0,103.0,5.0,7.0
1,143476.0,2012.0,2000.0,81.0,5.0,5.0
2,97676.0,2010.0,1995.0,85.0,5.0,5.0
3,111970.0,2004.0,1200.0,47.0,5.0,5.0
4,128886.0,2004.0,1200.0,47.0,5.0,5.0
...,...,...,...,...,...,...
3552907,54000.0,2013.0,1200.0,63.0,4.0,5.0
3552908,0.0,2000.0,1968.0,50.0,4.0,5.0
3552909,230000.0,2006.0,1900.0,100.0,4.0,5.0
3552910,0.0,2001.0,1968.0,103.0,4.0,5.0


In [102]:
num_data_tr.corrwith(clean_data.price_eur)

mileage               -0.000249
manufacture_year       0.000058
engine_displacement    0.002749
engine_power           0.000071
door_count            -0.000036
seat_count             0.000067
dtype: float64